<a href="https://colab.research.google.com/github/neiroYakoV/Ingate_SEO_n/blob/main/Yakovenko%20Vadim/%D0%BA%D0%BE%D0%BC_%D0%B7%D0%B0%D0%BF%D1%80_AutoKeras_28_%D0%B4%D0%B5%D0%BA%D0%B0%D0%B1%D1%80%D1%8F_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install autokeras
import autokeras as ak

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.8/584.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 60.0 MB/s eta 0:00:00
Using TensorFlow backend


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import requests
import zipfile
from io import BytesIO
from sklearn.model_selection import train_test_split
!pip install pandas openpyxl
from sklearn.datasets import load_files
import tensorflow as tf
from IPython.display import HTML, display
import os
import numpy as np

excel_file_path = '/content/drive/MyDrive/запросы.xlsx'
csv_file_path = '/content/drive/MyDrive/запросы.csv'

# Чтение Excel-файла
df = pd.read_excel(excel_file_path)

# Сохранение в формате CSV
df.to_csv(csv_file_path, encoding='utf-8', index=False)

# Загружаем данные обратно из CSV для проверки
df_from_csv = pd.read_csv(csv_file_path, encoding='utf-8')

# Выводим первые строки для проверки содержимого
print(df_from_csv.head())

    Поисковый запрос  Коммерциализация, %  Код  %коомерции 
0     купить минивэн                 1.00    1       4182.0
1   покупка опционов                 0.11    0       4518.0
2   заказать девочек                 0.92    1       8700.0
3  фотоальбомы оптом                 0.90    1          NaN
4        почему икаю                 0.00    0          NaN


In [ ]:
# Извлечение колонок "Поисковый запрос" и "Код"
queries = df_from_csv['Поисковый запрос'].tolist()
labels = df_from_csv['Код'].tolist()

# Преобразование меток в числовой формат: коммерческие - 1, некоммерческие - 0
#labels = [1 if label == 'коммерческие' else 0 for label in labels]



# Преобразование обработанных запросов и меток в массивы NumPy
x = np.array(queries)
y = np.array(labels)

# Разбиение набора данных на обучающую и валидационную выборки
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.15, random_state=42)

# Проверка размерностей массивов данных
print("Размерность x_train:", x_train.shape)
print("Размерность y_train:", y_train.shape)
print("Размерность x_val:", x_val.shape)
print("Размерность y_val:", y_val.shape)
print( x_train[2])
print( y_train[5:])

Размерность x_train: (7394,)
Размерность y_train: (7394,)
Размерность x_val: (1305,)
Размерность y_val: (1305,)
вакансия технолог
[1 0 0 ... 0 1 1]


> При обучении модели AutoKeras на текстах необходимо для задачи классификации использовать класс `TextClassifier`. Если используется "тяжелая" предобученная модель, например, bert, AutoKeras сам уменьшает параметр batch_size, не прерывая обучение.

In [ ]:
clf = ak.TextClassifier(
    overwrite=True,
    max_trials=3,
    objective = 'val_accuracy'
)

# Обучаем
hist=clf.fit(x_train,
             y_train, epochs=5,
             validation_data=(x_val, y_val),
             batch_size=64)

# Предсказание на валидационной выборке
predicted_y = clf.predict(x_val)

# Оценка точности модели
print(clf.evaluate(x_val, y_val))

Trial 3 Complete [01h 22m 44s]
val_accuracy: 0.8965517282485962

Best val_accuracy So Far: 0.9348658919334412
Total elapsed time: 01h 23m 37s
Epoch 1/5
116/116 [==============================] - 6s 40ms/step - loss: 0.4657 - accuracy: 0.7575 - val_loss: 0.1903 - val_accuracy: 0.9295
Epoch 2/5
116/116 [==============================] - 2s 15ms/step - loss: 0.1619 - accuracy: 0.9367 - val_loss: 0.1579 - val_accuracy: 0.9333
Epoch 3/5
116/116 [==============================] - 2s 14ms/step - loss: 0.0954 - accuracy: 0.9642 - val_loss: 0.1766 - val_accuracy: 0.9333
Epoch 4/5
116/116 [==============================] - 1s 12ms/step - loss: 0.0654 - accuracy: 0.9775 - val_loss: 0.1967 - val_accuracy: 0.9333
Epoch 5/5
41/41 [==============================] - 0s 5ms/step - loss: 0.2468 - accuracy: 0.9126
[0.24680748581886292, 0.9126436710357666]


In [ ]:
mod=clf.export_model()
mod.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None,)]                 0         
                                                                 
 expand_last_dim (ExpandLast  (None, 1)                0         
 Dim)                                                            
                                                                 
 text_vectorization (TextVec  (None, 512)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 512, 64)           320064    
                                                                 
 dropout (Dropout)           (None, 512, 64)           0         
                                                                 
 conv1d (Conv1D)             (None, 508, 256)          82176 